# Basic connectivity variables.
Only change if values of lab are modified.

In [3]:
# Basic connectivity variables.
# Only change if values of lab are modified.
router = "192.168.10.4"
port = 830
username = "vagrant"
password = "vagrant"
protocol = "ssh"

In [4]:
from ydk.providers import CodecServiceProvider, NetconfServiceProvider
from ydk.services import CRUDService, CodecService
# We need to create a provider to connect with the router. 
# There is a considerable theory behind this step (netconf, restconfg, etc.), 
# but we'll leave the details aside for this practice.
provider = NetconfServiceProvider(address=router,
                                      port=port,
                                      username=username,
                                      password=password, 
                                      protocol=protocol)
crud = CRUDService()

# YDK "binds" data from/to the router into python objects. 
# Still, we might want to look at the responses in xml for a quick. The next objects allow us to do that.

cd_provider = CodecServiceProvider(type="xml")
codec = CodecService()

ImportError: No module named ydk.providers

In [ ]:
# In case you want verbose logging, uncomment this lines.
#import logging
#log = logging.getLogger('ydk')
#log.setLevel(logging.DEBUG)
#ch = logging.StreamHandler()
#log.addHandler(ch)

# Read and modify the hostname
YDK simplifies many of the steps behind model-base device configuration. Let us start with a simple operation: reading and modifying the hostname of the device.

The model we'll use for this is Cisco-IOS-XR-shellutil-cfg. If you are curious of the content of the model, one can  download it from the device or find it at public repositories. This one in particular is in https://github.com/YangModels/yang/blob/master/vendor/cisco/xr/631/Cisco-IOS-XR-shellutil-cfg.yang.

Note: We could, more correctly, use Cisco-IOS-XR-shellutil-oper.yang for reading the current hostname. There is indeed a difference between oper and cfg Cisco XR (those changes are reflected on containers wihtin the models for IETF and Openconfig models). We'll ignore that point and leave that part for a more advanced example.

The model is part of the pre-compiled libraries of YDK. Reading the values of the model from the router requires us to create an object from the library and pass it as a "filter" to a read operation.

Using the libraries of YDK might feel uncomftable initially (or of any automatic generated library). However, it becomes more familiar with practice. 

In [4]:
# Load the shell config library
from ydk.models.cisco_ios_xr import Cisco_IOS_XR_shellutil_cfg as xr_shellutil_cfg

# Let us create an object. This will be used as filter for the read operation.

hostname=xr_shellutil_cfg.HostNames()

# Here we populated with the router data. 
# This will fail if any issue arises. Do not forget to implement proper error catching code in production.
hostname_populated = crud.read(provider, hostname)

In [5]:
# We can examine the response using the typical Jupyter facilities (e.g. autocomplete)
hostname_populated.

'test_XR'

In [6]:
# We can also see the response using the Codec objects we created before
print(codec.encode(cd_provider, hostname_populated))

<host-names xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-shellutil-cfg">
  <host-name>test_XR</host-name>
</host-names>



Let us modify the hostname of the router. What we do is to fill the object with the required values and commit it using a create (or update) operation.

In [5]:
new_hostname=xr_shellutil_cfg.HostNames()
new_hostname.host_name="Test"

crud.update(provider,new_hostname)
# No error here would mean that our transaction worked. Check it out in the router.

NameError: name 'xr_shellutil_cfg' is not defined

Try applying a illegal name for the router. See that an error comes and no change is commited.

# Obtaining a list of interfaces currently configured.
We can use the model Cisco_IOS_XR_ifmgr_cfg for this purpose. This will return information about the configured interfaces.
Note: I tried to use the operconfig model, which seem to be the more suitable for this case, but it did not work for me (and I needed to make progress). Always try to check with model is better for your case.

In [29]:
from ydk.models.cisco_ios_xr import Cisco_IOS_XR_ifmgr_cfg as xr_ifmgr_cfg
interface_configurations = xr_ifmgr_cfg.InterfaceConfigurations()  

In [35]:
# read data from NETCONF device
interfaces = crud.read(provider, interface_configurations)

In [31]:
# We can explore the data we obtained
interfaces.

<interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
  <interface-configuration>
    <active>act</active>
    <interface-name>MgmtEth0/RP0/CPU0/0</interface-name>
    <ipv4-network xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg">
      <addresses>
        <dhcp></dhcp>
      </addresses>
    </ipv4-network>
  </interface-configuration>
  <interface-configuration>
    <active>act</active>
    <interface-name>GigabitEthernet0/0/0/0</interface-name>
    <ipv4-network xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg">
      <addresses>
        <primary>
          <address>192.168.10.4</address>
          <netmask>255.255.255.0</netmask>
        </primary>
      </addresses>
    </ipv4-network>
  </interface-configuration>
</interface-configurations>



In [ ]:
# or visualize it
print(codec.encode(cd_provider, interfaces))

In [37]:
# Let us get a set of interfaces names
available_interfaces = set()
for interface_configuration in interfaces.interface_configuration:
    interface_name = interface_configuration.interface_name
    available_interfaces.add(interface_name)
print(available_interfaces)

set(['MgmtEth0/RP0/CPU0/0', 'GigabitEthernet0/0/0/0'])


# Create a loopback interface
There is no loopback in the router. We might need one. Let us create that as our first task.
A possible code to do that is  down. Try to do it first. Use the object we read before as an example.

Only one very important hit: you might need to create the interface first, then add the IP. There is a couple of non trivial things to do, so don't get discourage it if you don't get it right. 

In [ ]:
# Your solution goes here

In [8]:
# Or check the solution below
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#

In [40]:
# Create the interface first
from ydk.types import Empty
interface_configuration = interface_configurations.InterfaceConfiguration()
interface_configuration.active = "act"
interface_configuration.interface_name = "Loopback0"
interface_configuration.interface_virtual = Empty() 

In [41]:
# We can also see our requests, to make sure they are correct.
print(codec.encode(cd_provider, interface_configuration))

<interface-configuration xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
  <active>act</active>
  <interface-name>Loopback4</interface-name>
  <interface-virtual></interface-virtual>
</interface-configuration>



In [44]:
# Let us create the Interface
crud.create(provider,interface_configuration)

In [50]:
# Let us add the IP then
addresses = interface_configuration.ipv4_network.Addresses()
primary = addresses.Primary()
primary.address = "1.1.1.3"
primary.netmask = "255.255.255.255"
interface_configuration.ipv4_network.addresses.primary = primary

addresses.primary = primary

interface_configurations.interface_configuration.append(interface_configuration)

In [51]:
print(codec.encode(cd_provider, interface_configuration))

<interface-configuration xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
  <active>act</active>
  <interface-name>Loopback4</interface-name>
  <interface-virtual></interface-virtual>
  <ipv4-network xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg">
    <addresses>
      <primary>
        <address>1.1.1.3</address>
        <netmask>255.255.255.255</netmask>
      </primary>
    </addresses>
  </ipv4-network>
</interface-configuration>



In [52]:
# Let us create the Interface
crud.create(provider,interface_configuration)

In [7]:
# If not error was generated, we should have the Loopback ready. 
# We can repeat the interface listing code from before to test it is the case.